In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from itertools import product
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [4]:
#exten the all data
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [29]:
trainsets = pd.read_csv("/home/luming/kaggle/house-prices-advanced-regression-techniques/data/train.csv")
testsets = pd.read_csv("/home/luming/kaggle/house-prices-advanced-regression-techniques/data/test.csv")

In [79]:
#deal with the all dataset 
x_train=trainsets.iloc[:, :80].set_index('Id')
y_train=trainsets.iloc[:,-1]
x_test=testsets.iloc[:, :80].set_index('Id')

In [72]:
#import the variable sheet
vars_info=pd.read_csv("/home/luming/kaggle/house-prices-advanced-regression-techniques/data/variable_info.csv", index_col=0)
vars_info=vars_info.T
vars_info.head()
#because the SalePrice is Y, I drop the SalePrice.
vars_info.drop(columns=[80], axis=1)
vars_info.dropna(axis = 1)
vars_info=vars_info.T

In [73]:
#groups the data for corrleate or continuous
groups = {}
i = [0,1]
j = [0,1]
for x, y in product(i, j):
    tmp = vars_info[(vars_info.correlate == x).to_numpy() & (vars_info.continuous == y).to_numpy()]['Id'].to_numpy()
    groups[f'group_{x}_{y}'] = tmp


In [77]:
#clean data
